In [11]:
####################################Imports and Initial Feed Rip###################################################
import feedparser as fp, pandas as pd, datetime, time, string, requests as r
from bs4 import BeautifulSoup 
from functools import reduce

##TODO - Input these variables

rss_feed = 'http://feeds.huffingtonpost.com/c/35496/f/677097/index.rss'
company_info_df = pd.read_csv('Target_Company.csv')
company_name = company_info_df.iloc[0]['Company Name']
ticker_symbol = company_info_df.iloc[0]['Stock Symbol']




#Get the News Feed and Slice it to only include the news items
feed = fp.parse(rss_feed)
feed_items = feed['items']
###################################################################################################################

In [12]:
#################################Create the Cleaned News Items DataFrame###########################################

#Import the raw data into a DataFrame
raw_news_dataframe = pd.DataFrame(feed_items)

#Clean the DataFrame
def dataframe_cleaner(dataframe): 
    being_cleaned_df = pd.DataFrame(raw_news_dataframe[['author','id','title','summary']])
    
    for i in range(0,len(being_cleaned_df['summary'])):
        soup = BeautifulSoup(being_cleaned_df['summary'][i],'lxml')
        being_cleaned_df['summary'][i] = soup.get_text()
    
    return being_cleaned_df
    
#Create the cleaned Dataframe    
news_df = dataframe_cleaner(raw_news_dataframe)
news_df.head(1)

###################################################################################################################

,author,id,title,summary
0,Casey Williams,http://www.huffingtonpost.com/2016/04/07/meani...,The Dark Meaning Behind The Word 'Robot',Robots have arrived. They're sorting your pac...


In [13]:
#Load in the Sentiment DataFrame
sentiment_df = pd.read_csv('../Sentiment_Dictionary/sentiment_DataFrame.csv')
sentiment_df = sentiment_df.ix[:,'type':]
sentiment_df.head(1)

,type,word,position,stemmed,polarity
0,0.5,abandoned,adj,n,-1


In [14]:
##########################Perform Sentiment Analysis on the Title and Body of the Articles#########################
def title_analysis(title):
    tokenized_title = dict.fromkeys(title.split(' '))                          
                                
    for key in tokenized_title:
        tokenized_title[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    total_sentiment = sum([item for sublist in list(tokenized_title.values()) for item in sublist])
    normalized_sentiment = total_sentiment / len(tokenized_title)
    
    return normalized_sentiment

def summary_analysis(summary):
    tokenized_summary = dict.fromkeys(summary.split(' '))
    
    for key in tokenized_summary:
        tokenized_summary[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    total_sentiment = sum([item for sublist in list(tokenized_summary.values()) for item in sublist])
    normalized_sentiment = total_sentiment / len(tokenized_summary)
    return normalized_sentiment

def sentiment_analysis(dataframe):
    analyzing_df = dataframe
    analyzing_df['normalized_title_sentiment'] = list(map(lambda x: title_analysis(x),analyzing_df['title']))
    analyzing_df['normalized_summary_sentiment'] = list(map(lambda x: summary_analysis(x),analyzing_df['summary']))
    return analyzing_df

start_time = time.time()
analyzed_articles_df = sentiment_analysis(news_df)
elapsed_time = time.time() - start_time
print('Elapsed Time: ' + str(elapsed_time))
analyzed_articles_df.head(1)
analyzed_articles_df.to_csv('HuffingtonPostSentiment.csv')
###################################################################################################################

Elapsed Time: 442.2598078250885
